In [3]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, VotingRegressor, HistGradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression, ARDRegression
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor

In [27]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
def cleaner(df):
    colors = {'D' : 6, 'E' : 5, 'F' : 4, 'G' : 3, 'H' : 2, 'I' : 1, 'J' : 0}
    clarity = {'SI2' : 1, 'VVS2' : 5, 'VS2' : 3, 'VS1' : 4, 'SI1' : 2, 'VVS1' : 6, 'IF' : 7, 'I1' : 0}
    cut = {'Premium' : 4, 'Ideal' : 3, 'Very Good' : 2, 'Fair' : 0, 'Good' : 1}
    df['color'] = np.vectorize(colors.get)(df.color)
    df.clarity = np.vectorize(clarity.get)(df.clarity)
    df.cut = np.vectorize(cut.get)(df.cut)
    return df
train = cleaner(train)
test= cleaner(test)
X_train = train.drop(columns='price')
y_train = train.price
X_test = test

In [6]:
(-cross_val_score(HistGradientBoostingRegressor(), PolynomialFeatures().fit_transform(X_train), y_train, scoring='neg_root_mean_squared_error', cv=5)).mean()

0.09288154958730353

In [44]:

max_depth = [10]

learning_rate = [0.05,0.03,0.01]

max_iter=[200]

max_leaf_nodes=[None]

max_bins=[255]


In [45]:
random_grid={'max_iter': max_iter,
             'learning_rate': learning_rate,
             'max_depth': max_depth, 
             'max_leaf_nodes': max_leaf_nodes,
             'max_bins': max_bins}

In [47]:
r_search=RandomizedSearchCV(estimator=HistGradientBoostingRegressor(),                    # modelo
                             param_distributions=random_grid,   # dictio de hiperparametros
                             n_iter=3,                          # iteraciones
                             cv=4,                              # cross-validation
                             verbose=1,                         # que me de feedback
                             n_jobs=-1,                         # -1 es todos los nucleos
                             scoring='neg_root_mean_squared_error'                       # metrica de error
                             )

In [48]:
r_search.fit(PolynomialFeatures().fit_transform(X_train), y_train)

Fitting 4 folds for each of 3 candidates, totalling 12 fits


RandomizedSearchCV(cv=4, estimator=HistGradientBoostingRegressor(), n_iter=3,
                   n_jobs=-1,
                   param_distributions={'learning_rate': [0.05, 0.03, 0.01],
                                        'max_bins': [255], 'max_depth': [20],
                                        'max_iter': [200],
                                        'max_leaf_nodes': [None]},
                   scoring='neg_root_mean_squared_error', verbose=1)

In [39]:
two = r_search.best_estimator_.fit(PolynomialFeatures().fit_transform(X_train), y_train)

In [41]:
two_predict = two.predict(PolynomialFeatures().fit_transform(X_test))

In [50]:
(-cross_val_score(r_search.best_estimator_, PolynomialFeatures().fit_transform(X_train), y_train, scoring='neg_root_mean_squared_error', cv=5)).mean()

0.09174401684685493

In [42]:
send = pd.DataFrame(two_predict)
csv = send.rename(columns={0:'price'})
csv.index.name='id'
csv.to_csv('test8_OptiHGBR.csv')

In [49]:
r_search.best_estimator_

HistGradientBoostingRegressor(learning_rate=0.03, max_depth=20, max_iter=200,
                              max_leaf_nodes=None)

HistGradientBoostingRegressor(learning_rate=0.03, max_depth=10, max_iter=200,
                              max_leaf_nodes=None)

In [51]:
r2 = RandomForestRegressor(max_depth=20, min_samples_split=10, n_estimators=250)
r3 = HistGradientBoostingRegressor(learning_rate=0.03, max_depth=10, max_iter=200, max_leaf_nodes=None)
er = VotingRegressor([('rf', r2), ('hgb', r3)])

In [52]:
er.fit(PolynomialFeatures().fit_transform(X_train), y_train)

VotingRegressor(estimators=[('rf',
                             RandomForestRegressor(max_depth=20,
                                                   min_samples_split=10,
                                                   n_estimators=250)),
                            ('hgb',
                             HistGradientBoostingRegressor(learning_rate=0.03,
                                                           max_depth=10,
                                                           max_iter=200,
                                                           max_leaf_nodes=None))])

In [54]:
nine = er.predict(PolynomialFeatures().fit_transform(X_test))

In [55]:
send = pd.DataFrame(nine)
csv = send.rename(columns={0:'price'})
csv.index.name='id'
csv.to_csv('test9_Voting_OptiHGB_OptiRF.csv')

In [57]:
(-cross_val_score(er, X_train, y_train, scoring='neg_root_mean_squared_error', cv=5)).mean()

0.09091148814559886